In [ ]:
import sys
sys.path.append(r'../toollib_voro')

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np
import pandas as pd
from PointinPolygon import inpolygon
import os
from geopy.distance import great_circle
import ccfj
import h5py
from scipy import stats

In [ ]:
proj_name = 'output_FJSJ_06-05/'
outdir = proj_name + 'ds_vor/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

### Parameter settings

In [ ]:
# VoroTomo Partition times
#n = 100
n = 50
# number of Voro cells  range
kmin = 10
kmax = 40
# cell radius ?
Radius = 10 

In [ ]:
filename_basic = proj_name+'Basic_info.npy'
info_basic = np.load(filename_basic, allow_pickle='TRUE').item()      # setting dictionary
dir_stack_all = info_basic['dir_stack_all']
stalistname = info_basic['stalistname']
filename = dir_stack_all + 'gather_all.h5'

In [ ]:
info_basic['dir_ds_vor'] = outdir

In [ ]:
def Pairs(sta):
    p = []
    nsta = len(sta)
    for ii in range(nsta):
        for jj in range(ii+1,nsta):
            p.append([sta[ii],sta[jj]])
    return p
def cal_indx(pair,nsta):
    indx = int(pair[0]*(2*nsta-pair[0]-1)/2+pair[1]-pair[0]-1)
    return indx

def singleVoro(indx0,outname,xvyv):
    global ncfs
    global f
    global c
    global nf
    global r
    global count
    global outdir
    filename = os.path.join(outdir,outname+'.h5')
    if os.path.exists(filename):
        os.remove(filename)
    
    subpairs = Pairs(indx0)
    indx1 = [cal_indx(pair,nsta) for pair in subpairs]
    ncfsi = ncfs[indx1,:]
    counti = count[indx1]
    ri = r[indx1]
    indx = np.argsort(ri)
    ri = ri[indx]
    ncfsi = ncfsi[indx,:]
    counti = counti[indx]
    ncfsi = ncfsi[counti!=0]
    ri = ri[counti!=0]
    counti = counti[counti!=0]

    
    ds = ccfj.fj_noise(np.real(ncfsi),ri*1e3,c,f,itype=1,func=1)

    h5file=h5py.File(filename,'w')
    h5file.create_dataset('ds',data=ds)
    h5file.create_dataset('c',data=c)
    h5file.create_dataset('f',data=f)
    h5file.create_dataset('indx',data=indx0)
    h5file.create_dataset('xvyv',data=xvyv)
    h5file.close()

### Location information

In [ ]:
stainfo = pd.read_excel(stalistname+'.xlsx')
staloc =  np.asarray([stainfo.iloc[:,1],stainfo.iloc[:,2]]).T

### The KDE of the stations' locations

In [ ]:
kernel = stats.gaussian_kde(staloc.T)
nsta = len(stainfo.iloc[:,0])

### Read NCFs

In [ ]:
c = np.linspace(200,2000,1000)

In [ ]:
ncffile = h5py.File(dir_stack_all + 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count = ncffile['count'][:]
r = ncffile['r'][:]
ncffile.close()

### Voronoi partition

In [ ]:
x0 = np.mean(staloc[:,0])
y0 = np.mean(staloc[:,1])
minx = min(staloc[:,0]);maxx = max(staloc[:,0])
miny = min(staloc[:,1]);maxy = max(staloc[:,1])
points0 = []
for i in range(36):
    points0.append([x0+np.cos(i/18*np.pi),y0+np.sin(i/18*np.pi)])
    
points0=np.asarray(points0)

# Starting Voronoi Partitioning
for ii in range(n):
    # random the number of Voronoi cells
    k = np.random.randint(kmin,kmax)
    # Using the KDE PDF to generate Voronoi cells
    points = kernel.resample(k).T 
    # Using the Uniform distribution to generate Voronoi cells
    '''
    points = np.random.rand(k,2)
    points[:,0] = points[:,0]*(maxx-minx) + minx
    points[:,1] = points[:,1]*(maxy-miny) + miny
    '''

    # Voronoi Partition
    points = np.concatenate((points,points0))
    vor = Voronoi(points)
    areas = []
    for j in range(k):
        xv = vor.vertices[vor.regions[vor.point_region[j]],0]
        yv = vor.vertices[vor.regions[vor.point_region[j]],1]
        _in,_on = inpolygon(staloc[:,0],staloc[:,1],xv,yv)
        indx = np.array([i for i,x in enumerate(_in) if x])
        if len(indx)>10:
            areas.append([indx,np.array([xv,yv])])
    vor.close()
    for j in range(len(areas)):

        print(ii,j,len(areas[j][0]))
        # calculate the F-J spectrum
        singleVoro(areas[j][0],'vor'+str(ii)+'_'+str(j),areas[j][1])

In [ ]:
np.save(filename_basic,info_basic)